In [2]:
import os
os.getcwd()

'/Users/abhishek/codebases/dc-ml-extraction/notebooks/unsupervised_merchant_categorization'

In [3]:
import json
import time
from datetime import datetime, timedelta

import regex as re
import pandas as pd
import numpy as np

In [4]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
# INPUT_DATA_LOC = "/Users/abhishek/Downloads/717773d9-3f66-4bb4-b990-a07ad4e98a07.csv"
# INPUT_DATA_LOC = "/Users/abhishek/Downloads/a249663a-dd5d-4d4a-b564-8ef37804a8a2.csv"
INPUT_DATA_LOC = "/Users/abhishek/Downloads/5983ed36-7255-4b1f-82bd-53ec54225614.csv"

input_data = pd.read_csv(INPUT_DATA_LOC)
input_data.shape

(3103222, 5)

In [6]:
input_data.head()

,templatehash,servicename,pos,transactionnote,dt
0,0fde8791a26e98e372095005b02d0b39,HDFC,Rail Recipe,-1,2024-12-19
1,0fde8791a26e98e372095005b02d0b39,HDFC,GROWW INVEST TECH PRIVATE LIMITED,-1,2024-12-19
2,0fde8791a26e98e372095005b02d0b39,HDFC,NEXTBILLION TECHNOLOGY PRIVATE LIMITED,-1,2024-12-19
3,0fde8791a26e98e372095005b02d0b39,HDFC,RupeeRedee,-1,2024-12-19
4,0fde8791a26e98e372095005b02d0b39,HDFC,Sayyam Investments Private Limited,-1,2024-12-19


In [7]:
input_data.templatehash.nunique()

5074

In [8]:
input_data.servicename.value_counts(normalize=True) * 100

servicename
PAYTM                           7.782492
ICICI                           5.718540
HDFC                            5.625895
STATE BANK OF INDIA             4.972058
AXIS                            4.955366
                                  ...   
CO-OPERATIVE BANK OF MEHSANA    0.000032
CMSGICL                         0.000032
KHAMGAON URBAN BANK             0.000032
TP BANK                         0.000032
SHARAD BANK                     0.000032
Name: proportion, Length: 368, dtype: float64

In [9]:
input_data['transactionnote'].head(100)

0     -1
1     -1
2     -1
3     -1
4     -1
      ..
95    -1
96    -1
97    -1
98    -1
99    -1
Name: transactionnote, Length: 100, dtype: object

In [10]:
def wordify(text):
    '''
    Returns only alphabetical part from text
    '''
    pattern = r'[a-z]+'
    response = re.findall(pattern, str(text), re.IGNORECASE)
    response = " ".join([x.lower() for x in response if len(x) > 1])
    return response

In [11]:
for col in ['pos', 'transactionnote']:
    input_data[f'cleaned_{col}'] = input_data[col].parallel_apply(wordify)

In [13]:
input_data.head(3)

,templatehash,servicename,pos,transactionnote,dt,cleaned_pos,cleaned_transactionnote
0,0fde8791a26e98e372095005b02d0b39,HDFC,Rail Recipe,-1,2024-12-19,rail recipe,
1,0fde8791a26e98e372095005b02d0b39,HDFC,GROWW INVEST TECH PRIVATE LIMITED,-1,2024-12-19,groww invest tech private limited,
2,0fde8791a26e98e372095005b02d0b39,HDFC,NEXTBILLION TECHNOLOGY PRIVATE LIMITED,-1,2024-12-19,nextbillion technology private limited,


In [14]:
input_data.shape, input_data[input_data['cleaned_pos'] != ''].shape

((3103222, 7), (3083697, 7))

In [19]:
input_data = input_data[input_data['cleaned_pos'] != '']
input_data = input_data.drop_duplicates('cleaned_pos')
input_data.shape

(332199, 8)

In [20]:
input_data[['templatehash', 'cleaned_pos']].to_csv('input_data.csv', index=False)

In [21]:
input_data['pos_word_count'] = input_data['cleaned_pos'].parallel_apply(lambda x: len(x.split()))
input_data['pos_word_count'].value_counts(normalize=True) * 100

pos_word_count
2     30.864933
3     29.170166
4     16.760135
1     10.764331
5      7.753786
6      2.775144
7      0.885614
8      0.413909
9      0.197773
10     0.124925
11     0.082180
14     0.075858
12     0.045154
13     0.025587
15     0.022577
17     0.016857
16     0.015051
18     0.005117
19     0.000903
Name: proportion, dtype: float64

In [42]:
three_word_names = input_data.loc[input_data['pos_word_count'] == 3, ['cleaned_pos']]
three_word_names.head()

,cleaned_pos
22,iifl finance limited
64,lazypay repayment nbfc
80,www mpokket com
85,zerodha broking ltd
103,poonawalla fincorp ltd


In [24]:
two_word_names = input_data.loc[input_data['pos_word_count'] == 2, ['cleaned_pos']]
two_word_names.head()

,cleaned_pos
0,rail recipe
74,shravan ram
77,amazon india
90,jagat consturctions
91,target power


In [ ]:
# 📌 Extract first and second words
two_word_names['first_word'] = two_word_names['cleaned_pos'].apply(lambda x: str(x).split()[0])
two_word_names['second_word'] = two_word_names['cleaned_pos'].apply(lambda x: str(x).split()[1])

In [27]:
from collections import Counter

# 📊 Count occurrences of first and second words
first_word_counts = Counter(two_word_names["first_word"])
second_word_counts = Counter(two_word_names["second_word"])

In [28]:
# 📌 Convert to DataFrame for better analysis
first_word_df = pd.DataFrame(first_word_counts.items(), columns=["word", "count"]).sort_values(by="count", ascending=False)
second_word_df = pd.DataFrame(second_word_counts.items(), columns=["word", "count"]).sort_values(by="count", ascending=False)


In [29]:
first_word_df

,word,count
91,mr,1222
2275,upi,730
256,hotel,695
960,paypal,445
336,raz,405
...,...,...
16809,mrahmathulla,1
16808,nagoorgani,1
16807,apraj,1
16806,dhamdhamu,1


In [33]:
first_word_df['count'].describe(percentiles=[i/20 for i in range(10, 21)])

count    35797.000000
mean         2.864290
std         12.962967
min          1.000000
50%          1.000000
55%          1.000000
60%          1.000000
65%          1.000000
70%          2.000000
75%          2.000000
80%          2.000000
85%          3.000000
90%          4.000000
95%          8.000000
100%      1222.000000
max       1222.000000
Name: count, dtype: float64

In [59]:
first_word_df[first_word_df['count'] < 3]

,word,count
11028,medchal,2
11400,citizen,2
2276,snehanjali,2
17901,manvir,2
11387,dpk,2
...,...,...
16809,mrahmathulla,1
16808,nagoorgani,1
16807,apraj,1
16806,dhamdhamu,1


In [30]:
second_word_df

,word,count
6,enterprises,1699
16,traders,1007
58,petroleum,957
23,enterprise,764
180,singh,723
...,...,...
13858,santhoshkumar,1
13857,muthumariyamman,1
13856,mahalik,1
13855,kunjeshkumar,1


In [36]:
second_word_df['count'].describe(percentiles=[i/20 for i in range(10, 21)])

count    29971.000000
mean         3.421074
std         20.344871
min          1.000000
50%          1.000000
55%          1.000000
60%          1.000000
65%          1.000000
70%          1.000000
75%          2.000000
80%          2.000000
85%          3.000000
90%          4.000000
95%          9.000000
100%      1699.000000
max       1699.000000
Name: count, dtype: float64

In [58]:
second_word_df[second_word_df['count'] < 10].word.tolist()

['kri',
 'mahato',
 'sethi',
 'heaven',
 'petromart',
 'intern',
 'security',
 'rohit',
 'gasoline',
 'hea',
 'tvs',
 'mxxxxa',
 'cloud',
 'nd',
 'cap',
 'txxxxi',
 'marketplace',
 'infocom',
 'drive',
 'bo',
 'imaging',
 'chaurasiya',
 'ads',
 'praka',
 'bhagwan',
 'servo',
 'royal',
 'cybs',
 'apartment',
 'tyre',
 'bakes',
 'ibrahim',
 'xpress',
 'tp',
 'dis',
 'sonkar',
 'vishal',
 'sures',
 'oils',
 'medicine',
 'communicati',
 'solapur',
 'ramu',
 'drug',
 'raghav',
 'mumba',
 'halder',
 'yogesh',
 'hi',
 'arun',
 'metro',
 'export',
 'sahil',
 'thorat',
 'shrivastava',
 'exclusive',
 'kerala',
 'bali',
 'mohapatra',
 'ibl',
 'negi',
 'pack',
 'distribution',
 'multi',
 'apparel',
 'noi',
 'tirupur',
 'fashi',
 'sure',
 'ah',
 'infrastruct',
 'donation',
 'subh',
 'needs',
 'tnin',
 'naka',
 'icic',
 'puram',
 'raipur',
 'lax',
 'do',
 'gowda',
 'sugar',
 'foundry',
 'huf',
 'pand',
 'stat',
 'jagdish',
 'stone',
 'nilesh',
 'healthcar',
 'kan',
 'utilities',
 'dist',
 'shekhar',

In [ ]:
second_word_df[second_word_df['count'] > 20]

,word,count
6,enterprises,1699
16,traders,1007
58,petroleum,957
23,enterprise,764
180,singh,723
...,...,...
477,liquor,21
1646,nair,21
3802,car,21
7798,jat,21


In [50]:
high_freq_words = second_word_df.loc[second_word_df['count'] > 20, 'word'].tolist()